In [3]:
from sqlalchemy import (
    create_engine,
    
)

In [44]:
from llama_index.core import SQLDatabase
# from llama_index.llms.groq import Groq
import urllib.parse
import os
from dotenv import load_dotenv
load_dotenv()



db_user= 'abubakar@el-technology.com'
db_password= '*l%xyG6&KhvGrspi'
db_host= 'sgs-server.database.windows.net'
db_name= 'SGS_Database'

# Construct connection string
params = urllib.parse.quote_plus('Driver={ODBC Driver 18 for SQL Server};Server=tcp:sgs-server.database.windows.net,1433;Database=SocialMediaTestDB;Uid=abubakar@el-technology.com;Pwd=*l%xyG6&KhvGrspi;Encrypt=yes;TrustServerCertificate=no;ConnectionTimeout=60;Authentication=ActiveDirectoryPassword')
# SQLAlchemy connection string
connection_string = f'mssql+pyodbc:///?odbc_connect={params}'


In [45]:
## Test the connection with Databse
from sqlalchemy import create_engine, text

# Use the connection string to create a SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection and fetch some data
# Test the connection using raw SQL
# Execute query

with engine.connect() as connection:
    sql_query = text("SELECT * FROM SocialMediaData")  # Corrected SQL query
    result = connection.execute(sql_query)
    counter = 0
    for row in result:
        print(row)
        counter += 1

        if counter >= 5:
            break

(datetime.date(2022, 10, 1), 90627, 21, 190, 0, 0, 0, 0, 0, 0, 0, 0)
(datetime.date(2022, 10, 2), 90669, 42, 205, 2, 99, 124, 4, 5, 2, 12192, 20230)
(datetime.date(2022, 10, 3), 90696, 27, 309, 1, 116, 76, 0, 4, 1, 7899, 13107)
(datetime.date(2022, 10, 4), 90720, 24, 338, 1, 285, 153, 10, 8, 1, 26523, 44009)
(datetime.date(2022, 10, 5), 90737, 17, 418, 0, 0, 0, 0, 0, 0, 0, 0)


In [46]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
2
sql_database = SQLDatabase(engine, include_tables=["SocialMediaData"])

In [37]:
sql_database.dialect

'mssql'

In [47]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["SocialMediaData"], sample_rows_in_table_info=2)


In [49]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

api_key = "61cdcb9020c5433394f3be3887531f86"
azure_endpoint = "https://sgsopenai.openai.azure.com/"
api_version = "2023-07-01-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    #model="gpt-4",
    deployment_name="gpt-35-turbo-16k",
    #deployment_name="gpt-4",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-large",
    deployment_name="text-embedding-3-large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [50]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [56]:
# from llama_index.core.query_engine import NLSQLTableQueryEngine

# table_names = ["SocialMediaData"]

# def_sql_query_engine = NLSQLTableQueryEngine(
#     sql_database=sql_database,
#     tables=table_names, llm=llm
# )


In [38]:
from llama_index.core.prompts.prompt_type import PromptType

UPDATED_TEXT_TO_SQL_TMPL = (
    "You are an expert in querying a social media database, specifically focused on Facebook account insights." 
    "Given an input question related to Facebook profiles, create a syntactically correct MSSQL query to retrieve relevant information." 
    "Then, examine the results of the query and return the answer."
    "You can order the results by a relevant column to return the most "
    "interesting examples in the database.\n\n"
    "Never query for all the columns from a specific table, only ask for a "
    "few relevant columns given the question.\n\n"
    "Pay attention to use only the column names that you can see in the schema "
    "description. "
    "Be careful to not query for columns that do not exist. "
    "Pay attention to which column is in which table. "
    "Also, qualify column names with the table name when needed. "
    "You are required to use the following format, each taking one line:\n\n"
    "Question: Question here\n"
    "SQLQuery: SQL Query to run\n"
    "SQLResult: Result of the SQLQuery\n"
    "Answer: Final answer here\n\n"
    "Only use tables listed below.\n"
    "{schema}\n\n"
    "Question: {query_str}\n"
    "SQLQuery: "
)

UPDATED_TEXT_TO_SQL_PROMPT = PromptTemplate(
    UPDATED_TEXT_TO_SQL_TMPL,
    prompt_type=PromptType.TEXT_TO_SQL,
)

In [1]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import text_to_sql_prompt
from llama_index.core import PromptTemplate
from llama_index.core import PromptTemplate



table_names = ["SocialMediaData"]

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=table_names, llm=llm,
    text_to_sql_prompt=UPDATED_TEXT_TO_SQL_PROMPT, 
    
)

In [65]:
query_str = "What is the total engagement on October 4, 2022?"
response = sql_query_engine.query(query_str)
response

Response(response='The total engagement on October 4, 2022 is 285.', source_nodes=[NodeWithScore(node=TextNode(id_='96bd3fbc-2f6a-4a0c-81fc-c19b6eb6423e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(285,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'96bd3fbc-2f6a-4a0c-81fc-c19b6eb6423e': {}, 'sql_query': "SELECT Engagement\nFROM SocialMediaData\nWHERE Date = '2022-10-04'", 'result': [(285,)], 'col_keys': ['Engagement']})

In [91]:
query_str = "This is the facebook account insights for a user, your role is to analyse the last three months data and provide concise recommendation in not more than three bullet point on how to increase outreach and engagement. use specific numbers for your recommendation and refer to weak points highlighting with specific numbers the current issues"
response = def_sql_query_engine.query(query_str)
response

Response(response="Based on the Facebook account insights for the last three months, here are three concise recommendations to increase outreach and engagement:\n\n1. Increase the total number of fans by 10% by implementing targeted advertising campaigns and promoting the page to relevant audiences.\n2. Improve engagement by 15% by regularly posting high-quality content, including videos, images, and interactive posts that encourage user interaction.\n3. Boost estimated reach and impressions by 20% by utilizing Facebook's paid advertising options, such as sponsored posts and boosted content, to expand the reach of your posts to a wider audience.", source_nodes=[NodeWithScore(node=TextNode(id_='30e4ef18-3fb5-4aef-b39a-22b97c36ad62', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_sepera

In [93]:

query_str = "This is the facebook account insights for a user, your role is to analyse the last three months data and provide concise recommendation in not more than three bullet point on how to increase outreach and engagement. use specific numbers for your recommendation and refer to weak points highlighting with specific numbers the current issues"
response = sql_query_engine.query(query_str)
response

Response(response="Based on the analysis of the last three months' data, here are three concise recommendations to increase outreach and engagement:\n\n1. Increase the number of brand posts: The current average number of brand posts per month is 20. To improve outreach, aim to increase this number to at least 30 posts per month.\n\n2. Enhance engagement through comments and shares: The average number of comments and shares per post is relatively low at 5 and 10 respectively. Encourage more audience interaction by setting a target of at least 10 comments and 20 shares per post.\n\n3. Focus on organic reach: The estimated reach and impressions for organic content are significantly lower compared to overall reach. Allocate more resources towards organic content creation and promotion to increase organic reach by at least 20%.\n\nBy implementing these recommendations, we can address the weak points in the current strategy and improve outreach and engagement on the Facebook account.", sourc

In [94]:
print(response)

Based on the analysis of the last three months' data, here are three concise recommendations to increase outreach and engagement:

1. Increase the number of brand posts: The current average number of brand posts per month is 20. To improve outreach, aim to increase this number to at least 30 posts per month.

2. Enhance engagement through comments and shares: The average number of comments and shares per post is relatively low at 5 and 10 respectively. Encourage more audience interaction by setting a target of at least 10 comments and 20 shares per post.

3. Focus on organic reach: The estimated reach and impressions for organic content are significantly lower compared to overall reach. Allocate more resources towards organic content creation and promotion to increase organic reach by at least 20%.

By implementing these recommendations, we can address the weak points in the current strategy and improve outreach and engagement on the Facebook account.


In [90]:
print(response)

Based on the analysis of the last three months' data, here are the recommendations to increase outreach and engagement:

1. Increase the number of brand posts: The current average number of brand posts per month is 15. To improve outreach and engagement, aim to increase this number to at least 20 posts per month.

2. Enhance organic reach: The estimated reach for organic posts is currently 10,000. To expand outreach, focus on optimizing organic content and aim to increase the estimated reach to 15,000.

3. Encourage more shares and comments: The average number of shares and comments per post is 50 and 30 respectively. To boost engagement, aim to increase these numbers to at least 75 shares and 50 comments per post.

By implementing these recommendations, you can address the current weak points and significantly improve outreach and engagement on your Facebook account.


In [80]:
print(response)

Based on the Facebook account insights for the last three months, here are three concise recommendations to increase outreach and engagement:

1. Increase the total number of fans by 10% by implementing targeted advertising campaigns and promoting the page to relevant audiences.
2. Improve engagement by 15% by regularly posting high-quality and engaging content, such as videos, polls, and interactive posts.
3. Enhance the estimated reach by 20% by utilizing Facebook's ad targeting options to reach a wider audience and optimizing the timing of posts to maximize visibility.

The current weak points that need attention are as follows:
1. The total number of fans has only increased by 5% in the last three months, indicating a need for more effective strategies to attract new followers.
2. The engagement rate has decreased by 8% compared to the previous period, suggesting a need for more compelling and interactive content.
3. The estimated reach has remained stagnant, indicating a need for 

In [83]:
response.metadata

{'a039699a-2628-4a73-9af0-6ae189cd511d': {},
 'sql_query': 'SELECT Date, Total_Fans, Engagement, Estimated_Reach, Estimated_impressions\nFROM SocialMediaData\nWHERE Date >= DATEADD(month, -3, GETDATE())\nORDER BY Date DESC',
 'result': [],
 'col_keys': ['Date',
  'Total_Fans',
  'Engagement',
  'Estimated_Reach',
  'Estimated_impressions']}

In [59]:
query_str = "What is the total engagement on October 4, 2022?"

response = sql_query_engine.query(query_str)
response


Response(response='The total engagement on October 4, 2022 is 285.', source_nodes=[NodeWithScore(node=TextNode(id_='fea2ba92-3cc6-4235-8a8b-f7ea2a7f7f13', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(285,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'fea2ba92-3cc6-4235-8a8b-f7ea2a7f7f13': {}, 'sql_query': "SELECT Engagement\nFROM SocialMediaData\nWHERE Date = '2022-10-04'", 'result': [(285,)], 'col_keys': ['Engagement']})

In [68]:
query_str = "How many likes were there on October 3, 2022?"
response = sql_query_engine.query(query_str)
response

Response(response='There were 76 likes on October 3, 2022.', source_nodes=[NodeWithScore(node=TextNode(id_='dae4b3a9-bb9f-4968-a573-7621f9865b47', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(76,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'dae4b3a9-bb9f-4968-a573-7621f9865b47': {}, 'sql_query': "SELECT Likes\nFROM SocialMediaData\nWHERE Date = '2022-10-03'", 'result': [(76,)], 'col_keys': ['Likes']})

In [70]:
query_str = "What is the estimated reach on October 7, 2022?"

response = sql_query_engine.query(query_str)
response

Response(response='The estimated reach on October 7, 2022 is 0.', source_nodes=[NodeWithScore(node=TextNode(id_='3594ea10-7b12-4170-9ef7-002b692dd61c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(0,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'3594ea10-7b12-4170-9ef7-002b692dd61c': {}, 'sql_query': "SELECT Estimated_Reach\nFROM SocialMediaData\nWHERE Date = '2022-10-07'", 'result': [(0,)], 'col_keys': ['Estimated_Reach']})

In [72]:

query_str = "How many comments were made on October 4, 2022?"

response = sql_query_engine.query(query_str)
response

Response(response='There were 10 comments made on October 4, 2022.', source_nodes=[NodeWithScore(node=TextNode(id_='0ffb86b8-5a82-4a37-8fe1-7b83b91338a5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(10,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'0ffb86b8-5a82-4a37-8fe1-7b83b91338a5': {}, 'sql_query': "SELECT Comments\nFROM SocialMediaData\nWHERE Date = '2022-10-04'", 'result': [(10,)], 'col_keys': ['Comments']})

In [73]:
response.metadata

{'0ffb86b8-5a82-4a37-8fe1-7b83b91338a5': {},
 'sql_query': "SELECT Comments\nFROM SocialMediaData\nWHERE Date = '2022-10-04'",
 'result': [(10,)],
 'col_keys': ['Comments']}

In [76]:
query_str = "What is the total number of fans on October 1, 2022?"

response = sql_query_engine.query(query_str)
response


Response(response='The total number of fans on October 1, 2022 is 90,627.', source_nodes=[NodeWithScore(node=TextNode(id_='dcbbd45c-d67a-4a7c-99bf-beed0f7952be', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(90627,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'dcbbd45c-d67a-4a7c-99bf-beed0f7952be': {}, 'sql_query': "SELECT Total_Fans\nFROM SocialMediaData\nWHERE Date = '2022-10-01'", 'result': [(90627,)], 'col_keys': ['Total_Fans']})

In [78]:
query_str = "List all dates where the brand posted at least one post."

response = sql_query_engine.query(query_str)
response

Response(response='The brand posted at least one post on the following dates: October 2, 3, 4, 10, 11, 16, 17, 19, 20, 22, 23, 24, 26, 28, 30, and 31; November 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 26, 27, 28, 29, and 30; December 1, 4, 5, 6, 7, 8, 9, 12, 15, 16, 20, 21, 22, 25, 27, 28, and 29.', source_nodes=[NodeWithScore(node=TextNode(id_='f8c0b2aa-3099-420e-a197-9043e3c20f61', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(datetime.date(2022, 10, 2),), (datetime.date(2022, 10, 3),), (datetime.date(2022, 10, 4),), (datetime.date(2022, 10, 10),), (datetime.date(2022, 10, 11),), (datetime.date(2022, 10, 16),), (datetime.date(2022, 10, 17),), (datetime.date(2022, 10, 19),), (datetime.date(2022, 10, 20),), (datetime.date(2022, 10, 22),), (datetime.date(2022, 10, 23),), (datetime.date(2022, 10, 24),), (datetime.date(2022, 10, 26),), (datetime.date(2022, 10, 28),), (datetime.date(2022, 10, 30

In [30]:
print(response)

The brand posted at least one post on the following dates: October 2, October 3, October 4, October 10, October 11, October 16, October 17, October 19, October 20, October 22, October 23, October 24, October 26, October 28, October 30, October 31, November 3, November 6, November 8, November 9, November 10, November 11, November 12, November 13, November 14, November 15, November 16, November 17, November 19, November 21, November 22, November 23, November 24, November 26, November 27, November 28, November 29, November 30, December 1, December 4, December 5, December 6, December 7, December 8, December 9, December 12, December 15, December 16, December 20, December 21, December 22, December 25, December 27, December 28, and December 29.


In [31]:
response.metadata

{'ef8dc796-a2f7-4d9a-a6af-8e79cfe3fa87': {},
 'sql_query': 'SELECT Date\nFROM SocialMediaData\nWHERE Brand_Posts > 0\nORDER BY Date',
 'result': [(datetime.date(2022, 10, 2),),
  (datetime.date(2022, 10, 3),),
  (datetime.date(2022, 10, 4),),
  (datetime.date(2022, 10, 10),),
  (datetime.date(2022, 10, 11),),
  (datetime.date(2022, 10, 16),),
  (datetime.date(2022, 10, 17),),
  (datetime.date(2022, 10, 19),),
  (datetime.date(2022, 10, 20),),
  (datetime.date(2022, 10, 22),),
  (datetime.date(2022, 10, 23),),
  (datetime.date(2022, 10, 24),),
  (datetime.date(2022, 10, 26),),
  (datetime.date(2022, 10, 28),),
  (datetime.date(2022, 10, 30),),
  (datetime.date(2022, 10, 31),),
  (datetime.date(2022, 11, 3),),
  (datetime.date(2022, 11, 6),),
  (datetime.date(2022, 11, 8),),
  (datetime.date(2022, 11, 9),),
  (datetime.date(2022, 11, 10),),
  (datetime.date(2022, 11, 11),),
  (datetime.date(2022, 11, 12),),
  (datetime.date(2022, 11, 13),),
  (datetime.date(2022, 11, 14),),
  (datetime.d

In [81]:
query_str = "How many shares were there in total from October 1 to October 7, 2022?"

response = sql_query_engine.query(query_str)
response

Response(response='There were a total of 17 shares from October 1 to October 7, 2022.', source_nodes=[NodeWithScore(node=TextNode(id_='1b3e64a1-5700-434a-9989-425e4dea1f57', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(17,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'1b3e64a1-5700-434a-9989-425e4dea1f57': {}, 'sql_query': "SELECT SUM(Shares) \nFROM SocialMediaData \nWHERE Date >= '2022-10-01' AND Date <= '2022-10-07'", 'result': [(17,)], 'col_keys': ['']})

In [82]:
query_str = "What is the average estimated reach over the given period?"

response = sql_query_engine.query(query_str)
response

Response(response='The average estimated reach over the given period is 6,362.', source_nodes=[NodeWithScore(node=TextNode(id_='bb5fb2f8-0411-45c7-8ef0-aec0534b1f4d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(6362,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'bb5fb2f8-0411-45c7-8ef0-aec0534b1f4d': {}, 'sql_query': 'SELECT AVG(Estimated_Reach) AS Average_Estimated_Reach\nFROM SocialMediaData', 'result': [(6362,)], 'col_keys': ['Average_Estimated_Reach']})

In [96]:
response.metadata

{'0d78f891-9f7d-4efe-99b1-7cc9f11e691c': {},
 'sql_query': 'SELECT Likes\nFROM SocialMediaData',
 'result': [(0,),
  (124,),
  (76,),
  (153,),
  (0,),
  (0,),
  (0,),
  (0,),
  (0,),
  (184,),
  (111,),
  (0,),
  (0,),
  (0,),
  (0,),
  (125,),
  (94,),
  (0,),
  (204,),
  (242,),
  (0,),
  (46,),
  (169,),
  (189,),
  (0,),
  (53,),
  (0,),
  (45,),
  (0,),
  (51,),
  (119,),
  (0,),
  (0,),
  (109,),
  (0,),
  (0,),
  (99,),
  (0,),
  (147,),
  (37,),
  (38,),
  (43,),
  (20,),
  (73,),
  (156,),
  (54,),
  (109,),
  (54,),
  (0,),
  (15,),
  (0,),
  (12,),
  (84,),
  (70,),
  (63,),
  (0,),
  (29,),
  (129,),
  (27,),
  (31,),
  (84,),
  (75,),
  (0,),
  (0,),
  (76,),
  (26,),
  (32,),
  (49,),
  (57,),
  (20,),
  (0,),
  (0,),
  (42,),
  (0,),
  (0,),
  (21,),
  (55,),
  (0,),
  (0,),
  (0,),
  (29,),
  (119,),
  (27,),
  (0,),
  (0,),
  (58,),
  (0,),
  (111,),
  (86,),
  (36,),
  (0,),
  (0,)],
 'col_keys': ['Likes']}

In [35]:
query_str = "List all the dates and the corresponding number of new fans."

response = sql_query_engine.query(query_str)
response

Response(response='Here are the dates and the corresponding number of new fans:\n\n- October 1, 2022: 21 new fans\n- October 2, 2022: 42 new fans\n- October 3, 2022: 27 new fans\n- October 4, 2022: 24 new fans\n- October 5, 2022: 17 new fans\n- October 6, 2022: 28 new fans\n- October 7, 2022: 12 new fans\n- October 8, 2022: 26 new fans\n- October 9, 2022: 27 new fans\n- October 10, 2022: 42 new fans\n- October 11, 2022: 33 new fans\n- October 12, 2022: 16 new fans\n- October 13, 2022: 15 new fans\n- October 14, 2022: 37 new fans\n- October 15, 2022: 20 new fans\n- October 16, 2022: 14 new fans\n- October 17, 2022: 19 new fans\n- October 18, 2022: 17 new fans\n- October 19, 2022: 22 new fans\n- October 20, 2022: 28 new fans\n- October 21, 2022: 27 new fans\n- October 22, 2022: 7 new fans\n- October 23, 2022: 13 new fans\n- October 24, 2022: 27 new fans\n- October 25, 2022: 59 new fans\n- October 26, 2022: 26 new fans\n- October 27, 2022: 15 new fans\n- October 28, 2022: 28 new fans\n- O

In [84]:
query_str = "What is the total engagement over the period from October 1 to October 7, 2022?"

response = sql_query_engine.query(query_str)
response

Response(response='The total engagement over the period from October 1 to October 7, 2022 is 500.', source_nodes=[NodeWithScore(node=TextNode(id_='0cdaeae2-bfdd-4c6d-adc4-159c460966bc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(500,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'0cdaeae2-bfdd-4c6d-adc4-159c460966bc': {}, 'sql_query': "SELECT SUM(Engagement) AS TotalEngagement\nFROM SocialMediaData\nWHERE Date >= '2022-10-01' AND Date <= '2022-10-07'", 'result': [(500,)], 'col_keys': ['TotalEngagement']})

In [86]:
query_str = "tell me the date when there are the highest number of impressions."

response = sql_query_engine.query(query_str)
response

Response(response="I'm sorry, but there seems to be an error with the SQL query. Please check the syntax and try again.", source_nodes=[NodeWithScore(node=TextNode(id_='120023d5-d559-4d35-8a7c-95e8d83d7d03', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Error: Statement 'SELECT Date, Estimated_impressions\\nFROM SocialMediaData\\nORDER BY Estimated_impressions DESC\\nLIMIT 1;' is invalid SQL.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'120023d5-d559-4d35-8a7c-95e8d83d7d03': {}, 'sql_query': 'SELECT Date, Estimated_impressions\nFROM SocialMediaData\nORDER BY Estimated_impressions DESC\nLIMIT 1;'})

In [39]:
response.metadata

{'b0b15737-ffa4-4721-b3f6-67213e4206c9': {},
 'sql_query': 'SELECT Date, Estimated_impressions\nFROM SocialMediaData\nORDER BY Estimated_impressions DESC\nLIMIT 1;'}

In [87]:
query_str = "Calculate the total number of new fans and the average engagement for each day of the week (e.g., Monday, Tuesday)."

response = sql_query_engine.query(query_str)
response

Response(response='The total number of new fans and the average engagement for each day of the week are as follows:\n- Monday: 283 new fans, average engagement of 84\n- Tuesday: 271 new fans, average engagement of 67\n- Wednesday: 260 new fans, average engagement of 59\n- Thursday: 228 new fans, average engagement of 44\n- Friday: 196 new fans, average engagement of 17\n- Saturday: 187 new fans, average engagement of 39\n- Sunday: 185 new fans, average engagement of 11', source_nodes=[NodeWithScore(node=TextNode(id_='3e975231-91f3-4bc9-b5df-839d18f1f94d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(1, 283, 84), (3, 271, 67), (2, 260, 59), (4, 228, 44), (6, 196, 17), (5, 187, 39), (7, 185, 11)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'3e975231-91f3-4bc9-b5df-839d18f1f94d': {}, 'sql_

In [88]:
print(response)

The total number of new fans and the average engagement for each day of the week are as follows:
- Monday: 283 new fans, average engagement of 84
- Tuesday: 271 new fans, average engagement of 67
- Wednesday: 260 new fans, average engagement of 59
- Thursday: 228 new fans, average engagement of 44
- Friday: 196 new fans, average engagement of 17
- Saturday: 187 new fans, average engagement of 39
- Sunday: 185 new fans, average engagement of 11


In [36]:
print(response)

The total number of new fans and the average engagement for each day of the week are as follows:
- Monday: 283 new fans, average engagement of 84
- Tuesday: 271 new fans, average engagement of 67
- Wednesday: 260 new fans, average engagement of 59
- Thursday: 228 new fans, average engagement of 44
- Friday: 196 new fans, average engagement of 17
- Saturday: 187 new fans, average engagement of 39
- Sunday: 185 new fans, average engagement of 11
